# MQTT bridged LoRa networks - demo

Start a Mosquitto container first. For example:
- Use `codes\_demo\1_start_broker.sh` to start a Mosquitto container on Raspberry Pi.
- Config files are in `mqtt_config\mqtt`.
- set `allow_anonymous true` in `mqtt_config\mqtt\config\mosquitto.conf` to allow anonymous client.

## Getting Started
What this notebook does:  
- Using a client on PC
- List connected nodes
- Send messages to remote nodes:
 - Return results (read GPIOs)via RPC mechanism.
 - Write data to remote nodes (write GPIOs).
 - Execute arbitrary code on remote nodes.

In [ ]:
import os
import sys
import time
import json
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'micropython')))
 
import client
from collections import OrderedDict

### Start client

In [ ]:
the_client = client.Client()
the_client.start()

while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')

### List of nodes

In [ ]:
# SF8_NODE_EUI = 'f3d308fffe00'
# SF8_GATEWAY_EUI = '32aea4fffe809528'
# SF9_NODE_EUI = '32aea4fffe054928'
# SF9_GATEWAY_EUI = '260ac4fffe0c1764'

remote_nodes = ['32aea4fffe809528', '260ac4fffe0c1764']
remote_nodes

In [ ]:
targeted_node = '260ac4fffe0c1764'
targeted_node

### Prepare messages

In [ ]:
messages = OrderedDict()

___
## DEMOs
___

### Blink each node (broadcasting)

In [ ]:
messages['blink_led'] = {'message_type': 'command',
                         'command': 'blink led',
                         'kwargs': {'times': 3, 'on_seconds': 0.1, 'off_seconds': 0.1}}

the_client.request('Hub', messages['blink_led']);

### Send LoRa Packets

In [ ]:
# SF8_NODE_EUI = 'f3d308fffe00'
# SF8_GATEWAY_EUI = '32aea4fffe809528'
# SF9_NODE_EUI = '32aea4fffe054928'
# SF9_GATEWAY_EUI = '260ac4fffe0c1764'
pay_load = {"message": "LoRa packet sent via MQTT.", "time": int(time.time()), "to": "f3d308fffe00", "from": the_client.node.worker.name }
pay_load_json = json.dumps(pay_load) 
pay_load_json

In [ ]:
# transmit_payload: targeted_node will transmit the LoRa payload regardless of routing table.

messages['LoRa test'] = {'message_type': 'function',
                         'function': 'transmit_payload',
                         'kwargs': {'payload_string': pay_load_json}}
the_client.request(targeted_node, messages['LoRa test']);

In [ ]:
# dispatch_payload_json: 
    # find the nearest gateway to the destination node.
    # forward the payload data, and deligate to the nearest gateway to transmit the LoRa payload. 

messages['LoRa test'] = {'message_type': 'function',
                         'function': 'dispatch_payload_json',
                         'kwargs': {'pay_load_json': pay_load_json}}

the_client.request(targeted_node, messages['LoRa test']);

### Stop the demo

In [ ]:
# Stopping
the_client.stop()
the_client = None
print('\n[________________ Demo stopped ________________]\n')